<a href="https://colab.research.google.com/github/jdmartinev/IMDBAnalytics/blob/main/IMDBAnalyticsColab/IMDBAnalyticsColab_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial

En este tutorial, analizará una base de datos de películas *IMDb Movies dataset* para aprender conceptos y procedimientos básicos utilizados en el análisis de datos. Al final, usted tendrá la capacidad de:

-  Importar y explorar datos
-  Realizar limpieza de datos
-  Utilizar Pandas/Plotly para crear histogramas que le permitan analizar la distribución de los datos
-  Crear gráficos de líneas y de barras (_line charts_ & _bar charts_) para visualizar datos
-  Utilizar Pandas y Plotly para crear gráficas de dispersión (_scatter plots_) para realizar análisis de correlación


## Descarga de datos

Lo primero que hará será descargar los datos con los que va a trabajar:

- Tabla de películas (archivo csv).
- _Shapefile_ (archivo .shp y archivos auxiliares) con la información necesaria para graficar mapas. De forma resumida, este archivo almacena información geométrica en forma de líneas o polígonos. En este caso partícular, el _shapefile_ tendrá la geometría de los países que nos servirá para pintar en el mapa datos relevantes extraídos de la información contenida en la base de datos de películas.  

Estos archivos quedarán en la carpeta _/content/IMDBAnalyticsData/_ asociada con el sistema operativo del servidor en el que se está ejecutando google colab.


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1osH_xhTCW4Qh7f00VU_UaRK5whEXe8dr' -O data
!unzip "/content/data" -d "/content/IMDBAnalyticsData/"

### Importar las librerías necesarias

Importe las librerías que le servirán para el procesamiento y visualización de datos. La librería _geopandas_, que sirve para el procesamiento de archivos con información geográfica (_shapefiles_) no está instalada por defecto en el ambiente de google colab. Por esta razón, debe instalarla

In [ ]:
!pip install geopandas

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px

## **1. Procesamiento básico**

En este primer paso, cargará la base de datos (archivo .csv) a un _dataframe_ de pandas y utilizará algunas funciones de la librería que le ayudarán a tener información básica de los datos. De forma resumida, un _dataframe_ de pandas es una estructura de datos bidimensional en la que los datos se organizan en filas y columnas. Es similar a una hoja de cálculo en Excel, donde cada columna representa una variable y cada fila representa una observación.



In [ ]:
df = pd.read_csv('IMDBAnalyticsData/Data/movie_metadata.csv') #Cargar el archivo csv al dataframe df
df.head(5) #Mostrar las primeras 5 filas de la tabla 

In [ ]:
df.shape #Tamaño de la tabla (df.shape[0] es el número de filas y df.shape[1] es el número de columnas)

In [ ]:
df.dtypes #Muestra el tipo de dato que se almacena en cada columna de la tabla

Podemos acceder a las filas y columnas del _dataframe_ de las siguientes formas: 

- Acceder a todas las filas en la columna _director name_
```python 
df["director_name"] 
df.director_name
```
La segunda forma solo funciona si el nombre de la columna no tiene espacios

- Acceder a unas filas específicas de la columna _director name_
```python 
df.loc[0,"director_name"] 
df.loc[0:10,"director_name"]
```

- Acceder a filas y columnas específicas
```python 
df.loc[0:10,["director_name","movie_title"]]
df.iloc[0:10,[0,1]] 
```
El método ```loc``` indexa las columnas por su nombre, mientas que el método ```iloc``` indexa las columnas por su posición.

In [ ]:
df.loc[0:10,["director_name","movie_title"]]

In [ ]:
df.iloc[0:10,[0,1]] 

## **2. Preparación de datos**

Remueva algunas columnas que no son importantes para el análisis que se va a realizar.

- _movie_imdb_link_
- _num_critic_for_reviews_
- _genre_

El método ```drop``` permite hacer este proceso. El parámetro ```axis = 1``` se utiliza para que elimine información de las columnas, ```columns``` especifica las columnas a remover, e ```inplace = True``` se utiliza para que la modificación hecha se almacene en la varianle ```df``` sin necesidad de hacer una nueva asignación de variables. 


In [ ]:
df.drop(axis = 1, columns = ["movie_imdb_link","num_critic_for_reviews"],inplace = True)
df.head(5) #Mostrar de nuevo las 5 primeras filas para corroborar que se eliminaron las columnas 

Ejercicio: Remover la columna faltante

In [ ]:
#Código del ejercicio

Muestre las columnas como una lista para corroborar que las columnas no deseadas se eliminaron.

In [ ]:
list(df) 

Revise de nuevo el tamaño del dataframe. Debe ser diferente al tamaño inicial

In [ ]:
df.shape 

La columna _title_year_ almacena valores de tipo _float_. Sin embargo, los años se deben manejar como números enteros. Haga esta modificación utilizando el método ```astype```. Este método no permite utilizar el parámetro ```inplace```, por lo que es necesario hacer una re-asignación de la variable _df_.

In [ ]:
df['title_year'] = df['title_year'].astype("Int64")
df['title_year']

Cambie el nombre de la columna _gross_ a _movie_income_ utliando el método ```rename```

In [7]:
rename_dict = {"gross":"movie_income", "language":"Language"}
df.rename(columns = rename_dict,inplace = True)

## **3. Análisis de datos**

Para el análisis de datos utilizará diferentes tipos de figuras. Empezará con histogramas. Un histograma es una representación gráfica de la distribución de frecuencias de un conjunto de datos numéricos. En un histograma, se dividen los datos en una serie de "bins" o "intervalos", y se cuenta la frecuencia de observaciones que caen en cada bin. Luego, se grafica la frecuencia de cada bin en el eje vertical y el valor de cada bin en el eje horizontal, lo que permite visualizar la forma y la dispersión de la distribución de los datos.

En un histograma, la altura de cada barra representa la frecuencia de los datos que caen en ese bin. Los bins pueden ser de igual ancho (por ejemplo, cada bin puede representar un intervalo de 10 unidades) o de ancho variable (por ejemplo, para representar intervalos de mayor o menor densidad de datos).

Visualice la distribución de la columna _duration_ utilizando histogramas. Para esto utilizará el método ```hist```, el parámetro ```bins``` permitirá definir en cuántos intervalos se analizará la frecuencia de repeticiones de eventos, en este caso, duración de las películas. Se almacena la figura en la variable ```ax```, lo que permite agregar información adicional para su interpretación. en esta figura puede observar que la mayoría de películas duran aproximadamente 100 minutos.



In [ ]:
ax = df["duration"].hist(bins = 20)
ax.set_xlabel('Time [mins]')
ax.set_ylabel('Counts')

Hará la misma figura utilizando la librería _plotly express_

In [ ]:
fig = px.histogram(df, x="duration",nbins=40,labels={'duration':'Time [mins] '})
fig.show()

Ejercicio: Cree un histograma para ver la distribución de la columna _imdb_score_


In [ ]:
#Código del ejercicio

Puede hacer consultas con respecto a alguna variable. Por ejemplo, filtre las película que tengan un _imdb_score_ menor que 4

In [ ]:
df['imdb_score'] < 4
#df.imdb_score < 4 #Esta línea hace lo mismo

El resultado es un arreglo unidimensional de pandas de tipo _series_ (similar a un _dataframe_ pero con una sola columna) que va a tomar el valor ```True``` en las filas en las que la condición se cumple y ````False``` en las demás filas. Este nuevo arreglo se puede almacenar en una nueva variable o se puede utilizar para indexar el _dataframe_. En este caso, creará un nuevo _dataframe_ con nombre _df_low_score_ que tendrá todas las columnas de _df_ pero únicamente las filas que cumplen con la condición.  

In [ ]:
index = df['imdb_score'] < 4
df_low_score = df[index]
df_low_score

In [ ]:
#esta celda es una forma diferente de hacer lo mismo
df_low_score = df[df['imdb_score'] < 4]
df_low_score

Revise que en verdad las películas en este nuevo _dataframe_ tengan un _imdb_score_ menor a 4

In [ ]:
df_low_score.loc[:,['movie_title','imdb_score']]

Ejercicio: Revise la distribución de la columna _imdb_score_ de las películas producidas en USA utilizando histogramas. Si es necesario, cree _dataframes_ que le ayuden a resolver la tarea.

In [ ]:
#Código del ejercicio

El método ```groupby``` se usa para agrupar filas de un dataframe según los valores de una o varias columnas, y luego aplicar una o varias funciones de agregación a cada grupo. En otras palabras, el método ```groupby``` permite crear subconjuntos de datos de un dataframe basados en los valores de una o varias columnas, y luego realizar cálculos estadísticos sobre cada uno de estos subconjuntos.

Por ejemplo, si quiere saber qué años tienen el mayor y menor número de películas, podemos hacer lo siguiente:

In [ ]:
count_per_year = df.groupby('title_year').size()
print(count_per_year)
print(f"El año con mayor cantidad de películas producidas fue {count_per_year.idxmax()} con {count_per_year.max()} películas")
print(f"El año con menor cantidad de películas producidas fue {count_per_year.idxmin()} con {count_per_year.min()} películas")


El método ```size``` aplicado a una agrupación de filas creada con ```groupby``` retorna, en un arreglo de tipo ```series```, el número de elementos pertenecientes a cada uno de los grupos que se creó. En este caso particular, el número de películas producidas en cada año. En este arreglo, el índice corresponde a la variable en la cuál se creó el grupo (en este caso el año). 

El método ```max``` retorna el número más grande en el arreglo mientras que el método ```idxmax``` retorna el índice en el cuál se encuentra ese número.

Puede crear una gráfica de barras en la que se muestre la cantidad de películas producidas cada año de la siguiente forma



In [ ]:
ax = count_per_year.plot.bar(figsize = (18,10))
ax.set_xlabel('Year')
ax.set_ylabel('# of movies')

In [ ]:
fig = px.bar(x = count_per_year.index, y = count_per_year, labels={'y':'# of movies'})
fig.show()

Ejercicio: ¿En qué año se tiene el mayor y menor promedio de _imdb_score_?

Haga una gráfica de barras para mirar los promedios por año. Tip: El método ```mean()``` permite calcular promedios.

Quiere ahora saber en qué año se ha gastado más y en qué año se ha gastado menos presupuesto en películas.

Para esto, podemos hacer también una gráfica de líneas

In [ ]:
df_budget_year = df.groupby('title_year')['budget'].sum()
print(f"Se gastó más presupuesto en el año {df_budget_year.idxmax()}")


In [ ]:
ax = df_budget_year.plot()
ax.set_xlabel("Year")
ax.set_ylabel("Budget")

Se puede entender mejor la gráfica si mostramos el presupuesto en millones de dólares

In [ ]:
df_budget_year = df_budget_year / 100000

In [ ]:
fig = px.bar(x = df_budget_year.index, y = df_budget_year, labels={'y':'Budget (millions)'})
fig.show()


Ahora quiere saber si hay una relación entre  en _imdb_score_ de una película con su _movie_income_. Para esto, vamos a calcular la correlación entre las dos variables. 

La correlación es una medida estadística que describe la relación entre dos variables. En otras palabras, indica cómo se mueven o cambian juntas dos variables en relación a sus medias. Esta medida puede tomar valores entre -1 y 1. Un coeficiente de correlación de 1 indica una correlación positiva perfecta, lo que significa que a medida que aumenta una variable, también lo hace la otra en la misma proporción. Por otro lado, un coeficiente de correlación de -1 indica una correlación negativa perfecta, lo que significa que a medida que aumenta una variable, la otra disminuye en la misma proporción. Un coeficiente de correlación de 0 indica que no hay relación entre las dos variables.

La correlación la podemos analizar de forma visual o de forma cuantitativa.

Primero, calcule la correlación entre las dos variables, para esto, debe eliminar las filas con datos faltantes en estas variables.


In [ ]:
df_corr = df.loc[:,['budget','imdb_score']] #Dataframe con las variables a analizar
df_corr

In [ ]:
df_corr.dropna(inplace = True)
df_corr

El método ```dropna``` remueve las filas con valores faltantes, almacenados como ```nan``` en el _dataframe_. El parámetro ```inplace = True``` asegura que los cambios se almacenen en la variable _df_corr_.


El método ```plot.scatter``` grafica relaciones entre 2 variables. En este caso, en el eje x tenemos la variable _budget_ y en el eje y la variable _imdb_score_.

In [ ]:
ax = df_corr.plot.scatter('budget','imdb_score',figsize = (10,10))

Los valores atípico en la variable _budget_ dificultan la interpretación de la gráfica. Vamos a eliminar las filas del _dataframe_ con valores mayores a 2e9.

In [50]:
df_corr = df_corr[df_corr['budget']<2000000000]


In [ ]:
fig = px.scatter(df_corr, x="budget", y="imdb_score", labels = {'budget': 'Budget per movie', 'imdb_score':'IMDB score'})
fig.show()

Puede ver que hay una relación entre las dos variables. Si quiere cuantificar esta relación puede utilizar la correlación

In [ ]:
corr_mat = df_corr.corr()
print(corr_mat)
print(f"La correlación entre las variables budget e IMDB score es de {corr_mat.iloc[0,1]:.2f}")

## **4. Análisis geoespacial**

En este punto llevará los análisis que se han hecho a gráficas que nos puedan dar información acerca de los países donde se produjeron las películas. para esto utilizará la librería ```geopandas```

La función ```gpd.read_file``` carga un _dataframe_ de ```geopandas```. En este caso está seleccionando las columnas con la información relevante para las figuras y les está cambiando el nombre. La columna _geometry_ tiene la información de los polígonos que conforman cada país.

In [ ]:
shpfile = 'IMDBAnalyticsData/Data/Map/ne_10m_admin_0_countries.shp'
geo_df = gpd.read_file(shpfile)[['ADMIN', 'ADM0_A3', 'geometry']]
geo_df.columns = ['country', 'country_code', 'geometry']
geo_df.head(3)

En este caso, puede remover la fila correspondiente a la antártica dado que ocupa mucho espacio en el mapa y no es relevante.

In [ ]:
geo_df = geo_df.drop(geo_df.loc[geo_df['country'] == 'Antarctica'].index)
geo_df.plot(figsize=(20, 20), edgecolor='white', linewidth=1, color='lightblue')

Puede observar la lista de países que se encuentran en el mapa, que seguramente son más de los países de los que se tiene registro de películas (eso por ahora es irrelevante)

In [ ]:
geo_df['country']

Ahora, va a obtener información que se pueda graficar en el mapa. Por ejemplo, el número de películas producidas en cada país.

In [ ]:
df_mbc = df.groupby('country').size().to_frame('# of movies')
df_mbc

Ahora, va a combinar la información de ambas tablas. Básicamente, debe adjuntar la información contenida en el _shapefile_ a cada uno de los países para los que se tiene registro de películas. Para esto va a utilizar la función ```pd.merge```.

La función ```pd.merge``` se utiliza para combinar dos o más dataframes en función de una o varias columnas en común, similar a un join en SQL. El resultado de la operación es un nuevo dataframe que contiene todas las filas y columnas de los dataframes originales que cumplen con las condiciones de combinación especificadas.

La función pd.merge toma como entrada dos dataframes, junto con los argumentos que indican las columnas en común en las que se desea fusionar los dataframes. Existen varios tipos de combinaciones que se pueden especificar mediante el argumento "how", como "inner" (intersección), "outer" (unión), "left" (izquierda) y "right" (derecha).

Cuando se utiliza la función ```pd.merge```, se puede especificar un conjunto de columnas en las que se debe basar la combinación, utilizando el argumento "on". También se pueden especificar diferentes nombres de columnas para cada dataframe, utilizando los argumentos ```left_on``` y ```right_on```. En caso de que las columnas en común tengan nombres diferentes en los dataframes originales, se pueden utilizar los argumentos ```left_index``` y ```right_index``` para especificar que las combinaciones se realicen en los índices en lugar de en las columnas.




In [ ]:
df_mbc_shp = pd.merge(left=geo_df, right=df_mbc, how='left', left_on='country', right_on='country')
df_mbc_shp

Por último, grafique en el mapa la cantidad de películas producidas en cada país.


In [ ]:
fig = px.choropleth_mapbox(df_mbc_shp,
                           geojson=df_mbc_shp.geometry,
                           locations=df_mbc_shp.index,
                           color="# of movies",
                           mapbox_style="open-street-map",
                           opacity = 0.5,
                           zoom=8.5)
fig.show()